This notebook demonstrates the process of using AWS Sagemaker in script mode to train a simple classification model.

The dataset I will be using is the fashion mnist dataset.  And the script will be run in local model without using a actual Sagemaker notebook instance.

### Import packages

In [23]:
import sagemaker
import os
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist as mnist

# aws_env is my helper package that contains aws related settings
from aws_env import alias

sess = sagemaker.Session()
role = 'arn:aws:iam::%s:role/service-role/AmazonSageMaker-ExecutionRole-20210613T122808' % alias

### Download datasets

In [ ]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
fashion_mnist = tf.keras.datasets.fashion_mnist

os.makedirs("./data", exist_ok = True)

np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

In [7]:
!pygmentize mnist-trainer.py

import argparse, os
import numpy as np

import tensorflow
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam


class MyCallback(tensorflow.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('acc') > 0.998:
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True


def create_model(lr):
    model = tensorflow.keras.models.Sequential([
        tensorflow.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tensorflow.keras.layers.MaxPooling2D(2, 2),
        tensorflow.keras.layers.Flatten(),
        tensorflow.keras.layers.Dense(128, activation='relu'),
        tensorflow.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=Adam(lr=lr),
                  loss='categorical_crossentropy',
                  metrics=['acc'])
    return model


if __name__ == '__main__':

    parser = argparse.ArgumentPars

### Create the sagemaker estimator in local mode

In [41]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='mnist-trainer.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='local',
                          framework_version="1.12", 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={'epochs': 1}
                         )

Windows Support for Local Mode is Experimental


In [42]:
training_input_path = 'file://./data/training'
validation_input_path = 'file://./data/validation'
tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

Creating 6hx3ay9l71-algo-1-zbxqn ... 
Creating 6hx3ay9l71-algo-1-zbxqn ... done
Docker Compose is now in the Docker CLI, try `docker compose up`

Attaching to 6hx3ay9l71-algo-1-zbxqn
6hx3ay9l71-algo-1-zbxqn | 2021-06-13 10:03:36,476 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
6hx3ay9l71-algo-1-zbxqn | 2021-06-13 10:03:36,488 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
6hx3ay9l71-algo-1-zbxqn | 2021-06-13 10:03:37,170 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
6hx3ay9l71-algo-1-zbxqn | 2021-06-13 10:03:37,188 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
6hx3ay9l71-algo-1-zbxqn | 2021-06-13 10:03:37,197 sagemaker-containers INFO     Invoking user script
6hx3ay9l71-algo-1-zbxqn | 
6hx3ay9l71-algo-1-zbxqn | Training Env:
6hx3ay9l71-algo-1-zbxqn | 
6hx3ay9l71-algo-1-zbxqn | {
6hx3ay9l71-algo-1-zbxqn |     "additional_framework_parameters": {},
6hx3a

6hx3ay9l71-algo-1-zbxqn | _________________________________________________________________
6hx3ay9l71-algo-1-zbxqn | Layer (type)                 Output Shape              Param #   
6hx3ay9l71-algo-1-zbxqn | =================================================================
6hx3ay9l71-algo-1-zbxqn | conv2d (Conv2D)              (None, 26, 26, 32)        320       
6hx3ay9l71-algo-1-zbxqn | _________________________________________________________________
6hx3ay9l71-algo-1-zbxqn | max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
6hx3ay9l71-algo-1-zbxqn | _________________________________________________________________
6hx3ay9l71-algo-1-zbxqn | flatten (Flatten)            (None, 5408)              0         
6hx3ay9l71-algo-1-zbxqn | _________________________________________________________________
6hx3ay9l71-algo-1-zbxqn | dense (Dense)                (None, 128)               692352    
6hx3ay9l71-algo-1-zbxqn | ______________________________________________________

### Model will be saved in S3